# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to jinhui.li@pku.edu.cn and will expire on July 28, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1470620494.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.756256     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.804639     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [45]:
model_all['coefficients'].print_rows(18,4)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [15]:
import numpy as np
min=-1
min_l1=0
for l1 in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None,l2_penalty=0.,l1_penalty=l1,verbose=False)
    tmp = model.predict(validation)
    tmp = tmp - validation['price']
    tmp = tmp.to_numpy()
    tmp = tmp.dot(tmp)
    if min==-1:
        min = tmp
        min_l1 = l1
    elif tmp<min:
        min = tmp
        min_l1 = l1
    print l1,tmp
print min_l1,min
    

10.0 6.25766285142e+14
31.6227766017 6.25766285362e+14
100.0 6.25766286058e+14
316.227766017 6.25766288257e+14
1000.0 6.25766295212e+14
3162.27766017 6.25766317206e+14
10000.0 6.25766386761e+14
31622.7766017 6.25766606749e+14
100000.0 6.25767302792e+14
316227.766017 6.25769507644e+14
1000000.0 6.25776517727e+14
3162277.66017 6.25799062845e+14
10000000.0 6.25883719085e+14
10.0 6.25766285142e+14


In [37]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None,l2_penalty=0.,l1_penalty=10,verbose=False)
model.get('coefficients').print_rows(18,4)
print model['coefficients']['value'].nnz()

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [16]:
import numpy as np
min=-1
min_l1=0
for l1 in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None,l2_penalty=0.,l1_penalty=l1,verbose=False)
    tmp = model.predict(testing)
    tmp = tmp - testing['price']
    tmp = tmp.to_numpy()
    tmp = tmp.dot(tmp)
    if min==-1:
        min = tmp
        min_l1 = l1
    elif tmp<min:
        min = tmp
        min_l1 = l1
    print l1,tmp
print min_l1,min

10.0 1.56983602382e+14
31.6227766017 1.56983602351e+14
100.0 1.56983602254e+14
316.227766017 1.56983601949e+14
1000.0 1.56983600982e+14
3162.27766017 1.56983597925e+14
10000.0 1.5698358826e+14
31622.7766017 1.56983557704e+14
100000.0 1.56983461152e+14
316227.766017 1.56983156601e+14
1000000.0 1.56982201232e+14
3162277.66017 1.56979257164e+14
10000000.0 1.56972779669e+14
10000000.0 1.56972779669e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [24]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None,l2_penalty=0.,l1_penalty=10000000.0,verbose=False)
model.get('coefficients').print_rows(18,4)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None |   19604.7163508    |  None  |
|     bedrooms     |  None |   7922.62023075    |  None  |
| bedrooms_square  |  None |   928.112147889    |  None  |
|    bathrooms     |  None |    25392.743443    |  None  |
|   sqft_living    |  None |   39.1083048767    |  None  |
| sqft_living_sqrt |  None |   1123.95233925    |  None  |
|     sqft_lot     |  None | -0.000823987151992 |  None  |
|  sqft_lot_sqrt   |  None |   147.859054391    |  None  |
|      floors      |  None |    21154.353574    |  None  |
|  floors_square   |  None |    12871.042877    |  None  |
|    waterfront    |  None |   598538.378522    |  None  |
|       view       |  None |   93089.3048849    |  None  |
|    condition     |  None |   6584.64358167    |  None  |
|      grade       |  None |   6200.37319739    |  None 

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [26]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [27]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [29]:
import numpy as np
min=-1
min_l1=0
li=[]
for l1 in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None,l2_penalty=0.,l1_penalty=l1,verbose=False)
    tmp = model.predict(testing)
    tmp = tmp - testing['price']
    tmp = tmp.to_numpy()
    tmp = tmp.dot(tmp)
    if min==-1:
        min = tmp
        min_l1 = l1
    elif tmp<min:
        min = tmp
        min_l1 = l1
    print l1,tmp
    li.append(model['coefficients']['value'].nnz())
print min_l1,min
print ''
print l1_penalty_values
print li

100000000.0 1.56998193279e+14
127427498.57 1.57061573462e+14
162377673.919 1.57147654144e+14
206913808.111 1.57318167494e+14
263665089.873 1.576395411e+14
335981828.628 1.58126272294e+14
428133239.872 1.58905814086e+14
545559478.117 1.60202272805e+14
695192796.178 1.62325904157e+14
885866790.41 1.65400999406e+14
1128837891.68 1.69935261645e+14
1438449888.29 1.76962594541e+14
1832980710.83 1.88644538887e+14
2335721469.09 2.04434504733e+14
2976351441.63 2.2542066459e+14
3792690190.73 2.5125896725e+14
4832930238.57 2.89400419739e+14
6158482110.66 3.22281825796e+14
7847599703.51 2.86765188562e+14
10000000000.0 2.86162037257e+14
100000000.0 1.56998193279e+14

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [38]:
l1_penalty_min = 2.97635144e+09
l1_penalty_max = 3.79269019e+09
print l1_penalty_min/1000000

2976.35144


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [32]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [41]:
import numpy as np
min=-1
min_l1=0
li=[]
for l1 in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None,l2_penalty=0.,l1_penalty=l1,verbose=False)
    tmp = model.predict(validation)
    tmp = tmp - validation['price']
    tmp = tmp.to_numpy()
    tmp = tmp.dot(tmp)
    n = model['coefficients']['value'].nnz()
    if min==-1 and n==max_nonzeros:
        min = tmp
        min_l1 = l1
    elif tmp<min and n==max_nonzeros:
        min = tmp
        min_l1 = l1
    print l1,tmp
    li.append(model['coefficients']['value'].nnz())
print ''
print 'min:'
print min_l1,min
print li

2976351440.0 9.66925692362e+14
3019316637.37 9.74019450085e+14
3062281834.74 9.81188367942e+14
3105247032.11 9.89328342459e+14
3148212229.47 9.98783211266e+14
3191177426.84 1.00847716702e+15
3234142624.21 1.01829878055e+15
3277107821.58 1.02824799221e+15
3320073018.95 1.03461690923e+15
3363038216.32 1.03855473594e+15
3406003413.68 1.04323723787e+15
3448968611.05 1.04693748875e+15
3491933808.42 1.05114762561e+15
3534899005.79 1.05599273534e+15
3577864203.16 1.06079953176e+15
3620829400.53 1.0657076895e+15
3663794597.89 1.06946433543e+15
3706759795.26 1.07350454959e+15
3749724992.63 1.07763277558e+15
3792690190.0 1.08186759232e+15

min:
3448968611.05 1.04693748875e+15
[10, 10, 10, 10, 10, 10, 10, 10, 8, 8, 8, 7, 7, 7, 7, 6, 6, 6, 6, 6]


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [44]:
l1=2976351440.0
print l1/1000000
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None,l2_penalty=0.,l1_penalty=3448968611.05,verbose=False)
model['coefficients'].print_rows(18,4)

2976.35144
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement 